<a href="https://colab.research.google.com/github/tkasod2/TIL/blob/main/statistics/statistic_remind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import sklearn
from scipy import stats
import numpy as np

## CAC(Customer Acquisition Cost) - 마케팅 비용과 Acquisition의 관계 분석
- pearson correlation coefficient

In [27]:
marketing_costs = [352, 164, 210, 425, 503, 232, 321, 556, 464, 578, 612, 434] # 집행된 유튜브 광고 비용 (단위 : 10,000원)
user_acquired = [7214, 6122, 6896, 8020, 10982, 9021, 9240, 10210, 9987, 11521, 9792, 9852] # (단위 : 명)

ad_df = pd.DataFrame({'Marketing_Costs':marketing_costs, 
                      'User_Acquired':user_acquired})
ad_df

,Marketing_Costs,User_Acquired
0,352,7214
1,164,6122
2,210,6896
3,425,8020
4,503,10982
5,232,9021
6,321,9240
7,556,10210
8,464,9987
9,578,11521


In [33]:
CAC = ad_df['Marketing_Costs'].sum()/ad_df['User_Acquired'].sum()
CAC*1e4
# 신규고객 유치 비용 446원
# 그런데 산술적으로만 생각해도 될까?

445.6305060767796

In [34]:
## pearson 상관계수
ad_df.corr()
## 마케팅 비용과 Acquisition의 관계는 강하게 존재한다

,Marketing_Costs,User_Acquired
Marketing_Costs,1.000000,0.803578
User_Acquired,0.803578,1.000000


In [35]:
stats.pearsonr(ad_df['Marketing_Costs'], ad_df['User_Acquired'])
## h0 : 관계 없음, h1: 관계 있음 
## p-values가 0.00164 이므로, 0.01의 유의수준 내에서도 의미가 있다라고 판단할 수 있음

(0.803577506954685, 0.0016386012345537468)

## Web a와 b의 체류시간 차이가 있는지에 대한 통계학적 검정 ( with. t-test)

In [14]:
# 체류시간
web_a = pd.DataFrame([20.5, 12.6, 19.5, 18.8, 13.4, 13.5, 17.5, np.nan, 12.8, 17.8, np.nan, 23.1, 10.6, np.nan, 11.5], 
                     columns=['Duration_A'])
web_b = pd.DataFrame([11.8, 10.7, np.nan, 12.5, np.nan, 14.9, 12.1, 13.9, 10.3, 9.0, 13.3, 12.4, 12.5], 
                     columns=['Duration_B'])

In [15]:
pd.concat([web_a, web_b], axis=1)

,Duration_A,Duration_B
0,20.5,11.8
1,12.6,10.7
2,19.5,NaN
3,18.8,12.5
4,13.4,NaN
5,13.5,14.9
6,17.5,12.1
7,NaN,13.9
8,12.8,10.3
9,17.8,9.0


In [19]:
# 체류시간 A > B인지 테스트 해보자
print(web_a.mean(), web_a.count())
print(web_b.mean(), web_b.count())

Duration_A    15.966667
dtype: float64 Duration_A    12
dtype: int64
Duration_B    12.127273
dtype: float64 Duration_B    11
dtype: int64


In [24]:
# 독립표본 t-test (두 집단간의 평균차이를 검정)
# h0 : 평균의 차이가 없음
# h1 : 평균 차이가 있음
stats.ttest_ind(web_a["Duration_A"].dropna().values,
                web_b["Duration_B"].dropna().values,
                equal_var=False) # A, B의 분산을 같다고 가정할지, 그러지 않을지
                # False를 쓴건, Welch's t-test를 쓰기 위함(True면, standard independent 2 sample test)
                # 이유는 
                # web_a와 web_b의 갯수가 다르고,
                # 동일 분산이 아니라는 가정이 합리적이기 때문


Ttest_indResult(statistic=3.0165632092150694, pvalue=0.008734970056646718)

## CTR(Click Through Rate)의 차이가 있는지에 대한 검정
- 빈도니까, Chi-square test  

![image](https://t1.daumcdn.net/cfile/tistory/993B734A5D92C71221)
![image](https://t1.daumcdn.net/cfile/tistory/998B3C3E5D92D1482F)


In [2]:
a_clicked = 144 # 버튼(배너) 시안 A를 누른 유저의 수
a_unclicked = 2362 # 버튼(배너) 시안 A를 보았으나 누르지 않은 유저의 수

b_clicked = 212 # 버튼(배너) 시안 B를 누른 유저의 수
b_unclicked = 2528 # 버튼(배너) 시안 B를 보았으나 누르지 않은 유저의 수

click_df = pd.DataFrame({'Clicked':[a_clicked, b_clicked], 
                        'Unclicked':[a_unclicked, b_unclicked]}, index=['Button_A', 'Button_B'])

In [4]:
conversion_rate = click_df['Clicked'] / (click_df['Clicked'] + click_df['Unclicked']) * 100
print("Button(or Banner Ad) A's Conversion Rate(or CTR) is : {:0.2f}%".format(conversion_rate[0]))
print("Button(or Banner Ad) B's Conversion Rate(or CTR) is : {:0.2f}%".format(conversion_rate[1]))

Button(or Banner Ad) A's Conversion Rate(or CTR) is : 5.75%
Button(or Banner Ad) B's Conversion Rate(or CTR) is : 7.74%


In [12]:
# Contingency table
click_df

,Clicked,Unclicked
Button_A,144,2362
Button_B,212,2528


In [25]:
## Chi-square 검정
# 관찰된 빈도가 기대되는 빈도와 유의미하게 다른지의 여부를 검정하기 위해 사용되는 검정방법
stats.chi2_contingency([click_df['Clicked'], click_df['Unclicked']])


(7.890855736993352,
 0.004968535119697213,
 1,
 array([[ 170.06023637,  185.93976363],
        [2335.93976363, 2554.06023637]]))